# Q 1. Retrieve a list of employees along with their corresponding customer names based on the orders they have processed 

```sql
select e.employee_id,e.first_name, e.last_name, o.order_id, c.company_name 
from orders as o, employees as e, customers as c
where o.customer_id = c.customer_id and 
o.employee_id = e.employee_id;


```
![](1.png)

# Q 2. a. Create a view that calculates the total sales for each product, including product names and total sales amounts. Show the contents in it.

```sql
CREATE OR REPLACE VIEW total_sales AS
SELECT p.product_name, sum(od.quantity)
FROM products as p, order_details as od
where od.product_id = p.product_id
group by p.product_name;

SELECT * FROM total_sales;


```

![](2a.png)

# Q 2.b Create a view to identify the top 10 customers based on the number of orders they placed and list their names and total order counts. 

```sql
CREATE OR REPLACE VIEW top10_customers AS
SELECT c.company_name, count(distinct od.order_id)
FROM customers as c, order_details as od, orders as o
where o.customer_id = c.customer_id and o.order_id = od.order_id 
group by c.company_name
order by count(distinct od.order_id) DESC
limit 10;

SELECT * FROM top10_customers;

```

![](2b.png)

# 2.c Pick random 5 products from the view created in (a) and show which of these products are purchased by these top customers and which suppliers provide them. The output should include the Customer Name, Total Orders, Product Name, Total Sales, and Supplier Contact Name.

```sql
with random_products as 
(
    select product_name
    from total_sales
    order by RANDOM()
    limit 5
)

select c.company_name as customer_name,
    count(distinct o.order_id) as total_orders,
    rp.product_name,
    ts.sum as total_sales,
    s.contact_name as supplier_contact_name
from random_products as rp
join order_details od on rp.product_name = (
    select p.product_name from products as p where p.product_id = od.product_id
)
join orders as o on o.order_id = od.order_id
join customers c on c.customer_id = o.customer_id
join top10_customers tc on tc.company_name = c.company_name
join products p on p.product_name = rp.product_name
join suppliers s on s.supplier_id = p.supplier_id
join total_sales ts on ts.product_name = rp.product_name
group by c.company_name, rp.product_name, ts.sum, s.contact_name
group by c.company_name, rp.product_name;

```
![](2c.png)

# Q3. Create a view that displays the list of customers and the total number of orders they have placed
```sql
CREATE OR REPLACE VIEW customers_c AS
SELECT c.company_name, count(distinct od.order_id)
FROM customers as c, order_details as od, orders as o
where o.customer_id = c.customer_id and o.order_id = od.order_id 
group by c.company_name
order by count(distinct od.order_id) DESC;

SELECT * FROM customers_c;

```

![](3.png)

# Q 4. Use the view created in Part A to answer Part B
## a. Create a view in PostgreSQL that displays customer and supplier details for products categorized as 'Dairy Products'

```sql
CREATE VIEW customers_and_supplier AS
SELECT c.customer_id, s.supplier_id
from products as p, suppliers as s, order_details as od, orders as o, customers as c
where 
p.supplier_id = s.supplier_id and
p.product_id = od.product_id and
od.order_id = o.order_id and
o.customer_id = c.customer_id and 
p.category_id = (select category_id
from categories
where category_name = 'Dairy Products');

SELECT * FROM customers_and_supplier;

```
![](4a.png)

## b. Count how many different dairy products are supplied by each supplier

```sql
select p.supplier_id,  count(p.product_id)
from products as p
join customers_and_supplier as cs on p.supplier_id =cs.supplier_id 
group by p.supplier_id;

```
![](4b.png)

## Q5.Create a view to show the average unit price of products by supplier.
#### I assumed we have to calculate average price of product by distinct suppliers like there may be suppliers providing same product at different price.
<!-- but if we want to know only average price of all products for each supplier then just removing od.product_id from group by clause and select clause will work. -->

```sql

create view avgcount as
select p.supplier_id, od.product_id, avg(od.unit_price)
from products as p
inner join order_Details as od on od.product_id = p.product_id
group by p.supplier_id, od.product_id;

select * from avgcount;

```

![](5.png)

# Q6. Create a function that returns the total number of orders for a given customer ID.

```sql
CREATE OR REPLACE FUNCTION total_orders_by_customer(customer_id_param character varying)
RETURNS integer AS $$
DECLARE
    total_orders integer;
BEGIN
    SELECT COUNT(*)
    INTO total_orders
    FROM orders
    WHERE customer_id = customer_id_param;
    RETURN total_orders;
END;
$$ LANGUAGE plpgsql;

SELECT total_orders_by_customer('ALFKI');


```

![](6.png)

# Q7 Create a function that calculates the total revenue from orders for a specific product ID and shows it along with the product name 
```sql
CREATE OR REPLACE FUNCTION total_revenue_by_product(product_id_param integer)
RETURNS table(total_revenue numeric, product_ids integer) AS $$
BEGIN
   RETURN Query 
	SELECT sum(unit_price*quantity)::numeric, product_id_param as product_ids
  FROM order_details WHERE product_id = product_id_param;
END;
$$ LANGUAGE plpgsql;

SELECT total_revenue_by_product(51);


```

![](7.png)

# 8. (a) Given the employee_ id, create a function to get the total number of orders handled by that employee 

```sql
create or replace function total_orders_by_employee(employee_ids integer)
returns integer as 
$$
declare
    total_orders integer;
begin
    select count(*) 
	Into total_orders
    from orders
    where employee_id = employee_ids;
    return total_orders;
end;
$$ 
language plpgsql;
-- select max(employee_id) from employees;
select total_orders_by_employee(9);


```
![](8a.png)

# 8. (b) Create a function that retrieves the names of the employees who have handled the highest number of orders. Use the function created in (a) to get the order count

```sql
create or replace function get_employee_with_max_orders()
returns table (employee_name text, order_count integer) as
$$
begin
    return query
    select e.first_name || ' ' || e.last_name as employee_name, total_orders_by_employee(e.employee_id) as order_count
    from employees as e
    order by total_orders_by_employee(e.employee_id) desc
    limit 1;
end;
$$ 
language plpgsql;

select * from get_employee_with_max_orders();

```

![](8b.png)

# 9. Create a function that retrieves the total number of orders and the total revenue for each employee based in London for August 1996. 

```sql
create or replace function london_employees()
returns table (first_name character varying, last_name character varying, total_orders bigint, total_revenue double precision) 
as 
$$
begin
    return query
    select e.first_name, e.last_name, count(o.order_id) as total_orders,
    sum(od.unit_price * od.quantity) AS total_revenue
    from employees as e
    join orders as o on e.employee_id = o.employee_id
    join order_details as od on o.order_id = od.order_id
    where o.order_date between '1996-08-01' AND '1996-08-31'
    and e.city = 'London'
    group by e.employee_id;
end;
$$ 
language plpgsql;

select london_employees();

```

![](9.png)


# Q10 Create a view from the relation order_details which displays the order_id, quantity, product_id and unit_price. Try deleting a row from the order_details and check how the view appears now. Similarly, delete a row from the created view and show its effect on the order_details table. Write your observations with justification as text along with corresponding screenshots. 

```sql

create view od as
select order_id, product_id, quantity, unit_price
from order_details;
```
![](10a.png)

```sql
delete from order_details
where order_id = 10248 and product_id = 11;

select * from od;
```
![](10b.png)
![](10c.png)

```sql
delete from od
where order_id = 10248 and product_id = 42;
select * from od;
select * from order_details;
```
![](10d.png)
![](10e.png)



**Observation for Q. 10**
> 1. Deleting a row from order_details table directly affects the view od which underlies on it, since a view is essentially a saved in master table and reflects the current state of the underlying table.
>
> 2. Deleting a row from the view od also deleted the corresponding row from the order_details table, as views in Postgres sql are typically updatable.